In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Merge, TimeDistributed, concatenate, Bidirectional, Masking, RepeatVector
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import Callback
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU, Conv2D
from keras.callbacks import Callback, History
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, Masking
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
# import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler,MinMaxScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [0]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test_BDIfz5B.csv')
campaign_df= pd.read_csv('campaign_data.csv')

In [0]:
train_df= train.merge(campaign_df,how='left')
test_df= test.merge(campaign_df,how='left')

In [0]:
train_df.send_date=pd.to_datetime(train_df.send_date,format="%d-%m-%Y %H:%M")

In [0]:
# Form sentences for clicks per user, open per user, etc
sentences=train_df.groupby('user_id').is_click.apply(lambda x: list(x))

In [0]:
sentences_open = train_df.groupby('user_id').is_open.apply(lambda x: list(x))

In [0]:
train_df['communication_id'],invercom= pd.factorize(train_df.communication_type)

In [0]:
sentences_sec = train_df.groupby('user_id').no_of_sections.apply(lambda x: list(x))

In [0]:
sentences_com = train_df.groupby('user_id').no_of_images.apply(lambda x: list(x))

In [0]:
sentences_links = train_df.groupby('user_id').total_links.apply(lambda x: list(x))

In [0]:
sennew=zip(sentences, sentences_open,sentences_sec,sentences_com,sentences_links)

In [0]:
# make n_rows*n_timesteps* n_features vector for all sequences
sequences_ori=pad_sequences(pd.Series(sennew).apply(lambda x: zip(x[0],x[1],x[2],x[3])).tolist(),value=-1,padding='pre')

In [0]:
# filter those which have length of 1
sentences2=sentences[sentences.apply(lambda x: len(x))>1]

In [0]:
sentences_open2 = sentences_open[sentences_open.apply(lambda x: len(x))>1]

In [0]:
sentences_sec2= sentences_sec[sentences_sec.apply(lambda x: len(x)>1)]

In [0]:
sentences_com2= sentences_com[sentences_com.apply(lambda x: len(x)>1)]

In [0]:
sentences_links2= sentences_links[sentences_links.apply(lambda x: len(x)>1)]

In [0]:
sennew=zip(sentences2, sentences_open2,sentences_sec2,sentences_com2,sentences_links2)
# seqser=pd.concat((pd.Series(sennew).apply(lambda x: zip(x[0],x[1])).apply(lambda x: random.sample(x,len(x))),
# pd.Series(sennew).apply(lambda x: zip(x[0],x[1]))))
seqser=pd.Series(sennew).apply(lambda x: zip(x[0],x[1],x[2],x[3]))
sequences=pad_sequences(seqser.tolist(),value=-1,padding='pre')

In [149]:
sequences.shape

(151470, 20, 2)

In [150]:
sequences_ori.shape

(168236, 20, 2)

In [0]:
train_X= sequences[:,:-1]
train_Y = sequences[:,-1]

In [0]:
train_Y= train_Y[:,0]

In [178]:
train_X.shape

(151470, 19, 4)

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [179]:

# define nn model
# emdedding_size=500
# vocab_size=5502
keras_model2 = Sequential()
# keras_model2.add(Masking(mask_value=-1. ,input_shape=(None,2)))
# keras_model2.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
#                     weights=[pretrained_weights]))
# keras_model2/.add(LSTM(units=100,return_sequences=True))
# keras_model2.add(Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform",input_shape=(None,4)))
# keras_model2.add(MaxPooling1D())
# print keras_model2.output_shape
# keras_model2.add(Conv1D(300, 3, padding='valid',activation='relu',strides=1 ,input_shape=(None,4)))
# keras_model2.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Flatten())
# keras_model2.add(Dropout(0.2))
# model.add(Dense(150,activation='sigmoid'))
# model.add(Dropout(0.2))
keras_model2.add(Conv1D(250,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1,input_shape=(None,4)))
# we use max pooling:
# keras_model2.add(GlobalMaxPooling1D())
keras_model2.add((LSTM(units=100,return_sequences=False,recurrent_dropout=0.2)))
# keras_model2.add((GRU(units=100,return_sequences=False,recurrent_dropout=0.2)))
# keras_model2.add(Conv1D(300, 3, padding='valid',activation='relu',strides=2 ,input_shape=(None,4)))
# keras_model2.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Flatten())
print keras_model2.output_shape
# keras_model2.add(Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform"))
# print keras_model2.output_shape
# keras_model2.add(GlobalMaxPooling1D())
print keras_model2.output_shape

keras_model2.add(Dropout(0.2))
# keras_model2.add(LSTM(units=500,return_sequences=False))
keras_model2.add(Dense(units=100))
keras_model2.add(Dropout(0.2))
keras_model2.add(Dense(units=10))
keras_model2.add(Dropout(0.2))
# keras_model2.add(Dense(units=5))
# keras_model2.add(Dropout(0.2))

keras_model2.add(Dense(units=1))
print keras_model2.output_shape
keras_model2.add(Activation('sigmoid'))
keras_model2.compile(optimizer='Adamax', loss='binary_crossentropy',metrics=['accuracy','mse'])

(None, 250)
(None, 250)
(None, 1)


In [0]:
# train_X=np.concatenate((np.ones_like(train_X[:,0]).reshape(train_X.shape[0],-1,4)*-1,train_X),axis=1)

In [0]:
xtrain, xval, ytrain, yval = train_test_split(train_X, train_Y, test_size=0.1, random_state=7)

In [182]:
train_X.shape, xtrain.shape, xval.shape

((151470, 19, 4), (136323, 19, 4), (15147, 19, 4))

In [0]:
# filepath="../input/best-model/best.hdf5"
import pickle
filename_m= 'cnnlstm_'
pickle.dump(keras_model2.to_json(),open(filename_m+'.pkl','w'))
filepath=filename_m+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
ra_val = RocAucEvaluation(validation_data=(xval, yval), interval = 1)
callbacks_list = [checkpoint,ra_val]

In [0]:
sample_weight=np.concatenate((np.ones(train_X.shape[0]/3)*3,np.ones(train_X.shape[0]/3)*2,
                              np.ones(train_X.shape[0]/3)))

In [185]:
xtrain.shape

(136323, 19, 4)

In [ ]:
keras_model2.fit(xtrain,ytrain, epochs=19, validation_data=(xval,yval),
          batch_size=1024, callbacks = callbacks_list,verbose=1)

In [0]:
  from keras.models import model_from_json
# keras_model2= model_from_json(pickle.load(open('lstm2fea100.pkl')))
# keras_model2.load_weights('lstmcorrectorderweights-improvement-24-0.99.hdf5')

In [0]:
nnpred=keras_model2.predict(sequences_ori)

In [0]:
snn= sentences.reset_index()

In [164]:
snn.shape, nnpred.shape , sentences.shape

((168236, 2), (168236, 1), (168236,))

In [0]:
snn['pred']= nnpred

In [0]:
test_df['send_date']= pd.to_datetime(test_df.send_date,format="%d-%m-%Y %H:%M")

In [0]:
test_df['send_week']=test_df.send_date.dt.week
test_df['send_day']= test_df.send_date.dt.day
test_df['send_hour']= test_df.send_date.dt.hour
test_df['send_hour']=(test_df.send_hour/6).astype('int')
test_df['send_weekday']=test_df.send_date.dt.weekday
# pred_nn=test_df.merge(snn,how='left').groupby(['campaign_id','send_weekday']).pred.apply(lambda x: x.fillna(x.mean()))

In [0]:
pred_nn=test_df.merge(snn,how='left').groupby(['campaign_id','send_weekday',
                                               'send_hour']).pred.apply(lambda x: x.fillna((x.quantile(0.9))))

In [0]:
submission= pd.read_csv('sample_submission_4fcZwvQ.csv')
submission.is_click = pred_nn
submission.to_csv('submission_lstmcnn.csv',index=False)

In [6]:
import matplotlib as pd
pd.__version__

'2.2.2'